In [1]:
%pip install numpy scipy.io torch sklearn pennylane

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\users\tahir\onedrive\desktop\qc\.venv\lib\site-packages (2.0.2)



ERROR: Could not find a version that satisfies the requirement scipy.io (from versions: none)
ERROR: No matching distribution found for scipy.io


In [2]:
import os
import numpy as np
import scipy.io
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import pennylane as qml
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Loading Data (eeg brain data)

In [8]:
directory = 'seed_pre/drive-download-20241126T210900Z-001'
files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.mat')]
all_eeg_data = []
all_labels = []

# Assume labels are stored separately and need to be matched to EEG data
labels_per_file = []  # Store how many segments each file has for later label matching

for file in files:
    filename = os.path.basename(file)
    data = scipy.io.loadmat(file)
    
    if filename == 'label.mat':
        labels = data['label'].flatten()  # Store labels, assume one label per segment if matched correctly
        continue

    # Identify the prefix used for EEG data in this file
    eeg_keys = [key for key in data.keys() if '_eeg' in key]
    if not eeg_keys:
        print(f"No EEG data found in {filename}.")
        continue

    prefix = eeg_keys[0].split('_eeg1')[0]
    
    # Collect EEG data segments
    try:
        file_eeg_data = [data[f'{prefix}_eeg{i}'] for i in range(1, 16)]
        all_eeg_data.extend(file_eeg_data)
        labels_per_file.append(len(file_eeg_data))  # Assuming each segment should have a label
    except KeyError as e:
        print(f"KeyError for file {filename}: {e}")

# After collecting all EEG data, extend labels to match the number of segments
for count in labels_per_file:
    all_labels.extend(labels[:count])  # Extend labels based on the number of segments in each file

print(f"Loaded EEG data from {len(all_eeg_data)} segments.")
print(f"Loaded labels count: {len(all_labels)}")


MemoryError: 

Setting up a Quantum Model

In [5]:
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(2))
    qml.templates.BasicEntanglerLayers(weights, wires=range(2))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(2)]

weight_shapes = {"weights": (1, 2)}
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)


flattening/splitting data

In [7]:
# Assuming all_eeg_data is a list of numpy arrays where each array represents a trial
eeg_data_flattened = [trial.flatten() for subject in all_eeg_data for trial in subject]

# Make sure this matches the number of trials in all_eeg_data
labels_flattened = np.repeat(all_labels, [len(subject) for subject in all_eeg_data])
print("Total number of EEG segments (should match number of labels):", sum(len(subject) for subject in all_eeg_data))
print("Total number of labels:", len(all_labels))


ValueError: operands could not be broadcast together with shape (15,) (675,)

Data was preprocessed seperately (Fyp dataset)


In [4]:
# Cell 3: Normalize Data and Setup Dataloader
scaler = MinMaxScaler(feature_range=(-1, 1))
eeg_data_flattened = np.vstack(all_eeg_data)
eeg_data_normalized = scaler.fit_transform(eeg_data_flattened)

tensor_x = torch.Tensor(eeg_data_normalized)  # Transform to torch tensor
tensor_y = torch.Tensor(all_labels)

my_dataset = TensorDataset(tensor_x, tensor_y)
train_loader = DataLoader(my_dataset, batch_size=64, shuffle=True)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 47001 and the array at index 1 has size 46601